In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_02a_why_sqrt5")' FastaiNotebook_02a_why_sqrt5

Installing packages:
	.package(path: "/home/ubuntu/dev_swift/FastaiNotebook_02a_why_sqrt5")
		FastaiNotebook_02a_why_sqrt5
With SwiftPM flags: []
Working in: /tmp/tmp3pmmk73v/swift-install
Updating https://github.com/mxcl/Path.swift
Updating https://github.com/latenitesoft/NotebookExport
Updating https://github.com/JustHTTP/Just
Completed resolution in 2.11s
Compile Swift Module 'FastaiNotebook_02a_why_sqrt5' (5 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [ ]:
//export
import Path
import TensorFlow

In [ ]:
import FastaiNotebook_02a_why_sqrt5

In [ ]:
// export
public typealias TI = Tensor<Int32>

### Data

In [ ]:
var (xTrain,yTrain,xValid,yValid) = loadMNIST(path: Path.home/".fastai"/"data"/"mnist_tst", flat: true)

In [ ]:
let trainMean = xTrain.mean()
let trainStd  = xTrain.std()

In [ ]:
xTrain = normalize(xTrain, mean: trainMean, std: trainStd)
xValid = normalize(xValid, mean: trainMean, std: trainStd)

In [ ]:
let (n,m) = (xTrain.shape[0],xTrain.shape[1])
let c = yTrain.max().scalarized()+1
print(n,m,c)

60000 784 10


In [ ]:
let nHid = 50

In [ ]:
public struct MyModel: Layer {
    public var layer1: FADense<Float>
    public var layer2: FADense<Float>
    
    public init(nIn: Int, nHid: Int, nOut: Int){
        layer1 = FADense(nIn, nHid, activation: relu)
        layer2 = FADense(nHid, nOut)
    }
    
    @differentiable
    public func call(_ input: TF) -> TF {
        return input.sequenced(through: layer1, layer2)
    }
}

In [ ]:
var model = MyModel(nIn: m, nHid: nHid, nOut: Int(c))

In [ ]:
let pred = model(xTrain)

### Cross entropy loss

In [ ]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let exped = exp(activations) 
    return log(exped / exped.sum(alongAxes: -1))
}

In [ ]:
let smPred = logSoftmax(pred)

In [ ]:
yTrain[0..<3]

[5, 0, 4]


In [ ]:
(smPred[0][5],smPred[1][0],smPred[2][4])

▿ 3 elements
  - .0 : -2.2364676
  - .1 : -3.4331937
  - .2 : -2.5348387


There is no fancy indexing yet so we have to use gather to get the indices we want out of our softmaxed predictions.

In [ ]:
func nll<Scalar>(_ input: Tensor<Scalar>, _ target :TI) -> Tensor<Scalar> 
    where Scalar:TensorFlowFloatingPoint{
        let idx: TI = Raw.range(start: Tensor(0), limit: Tensor(60000), delta: Tensor(1))
        let indices = Raw.concat(concatDim: Tensor(1), [idx.expandingShape(at: 1), target.expandingShape(at: 1)])
        let losses = Raw.gatherNd(params: input, indices: indices)
        return -losses.mean()
    }

In [ ]:
nll(smPred, yTrain)

2.4565928


In [ ]:
time(repeating: 100){ let _ = nll(smPred, yTrain) }

average: 0.9669732999999999 ms,   min: 0.902235 ms,   max: 1.107671 ms


Simplify `logSoftmax` with log formulas.

In [ ]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - log(exp(activations).sum(alongAxes: -1))
}

In [ ]:
let smPred = logSoftmax(pred)

In [ ]:
nll(smPred, yTrain)

2.4565928


Use LogSumExp trick

In [ ]:
smPred.max(alongAxes: -1).shape

▿ TensorShape
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 1


In [ ]:
func logSumExp<Scalar>(_ x: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let m = x.max(alongAxes: -1)
    return m + log(exp(x-m).sum(alongAxes: -1))
}

In [ ]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - logSumExp(activations)
}

In [ ]:
let smPred = logSoftmax(pred)

In [ ]:
nll(smPred, yTrain)

2.4565928


In S4TF nll loss is combined with softmax in:

In [ ]:
let loss = softmaxCrossEntropy(logits: pred, labels: yTrain)
loss

2.4565928


In [ ]:
time(repeating: 100){ let _ = nll(logSoftmax(pred), yTrain)}

average: 1.14561623 ms,   min: 1.074932 ms,   max: 1.31902 ms


In [ ]:
time(repeating: 100){ let _ = softmaxCrossEntropy(logits: pred, labels: yTrain)}

average: 1.07110238 ms,   min: 0.967635 ms,   max: 1.222231 ms


## Basic training loop

Basically the training loop repeats over the following steps:
- get the output of the model on a batch of inputs
- compare the output to the labels we have and compute a loss
- calculate the gradients of the loss with respect to every parameter of the model
- update said parameters with those gradients to make them a little bit better

In [ ]:
// export
public func accuracy(_ output: TF, _ target: TI) -> TF{
    let corrects = TF(output.argmax(squeezingAxis: 1) .== target)
    return corrects.mean()
}

In [ ]:
print(accuracy(pred, yTrain))

0.101516664


In [ ]:
let bs=64                         // batch size
let xb = xTrain[0..<bs]          // a mini-batch from x
let preds = model(xb) //predictions
print(preds[0], preds.shape)

[  0.30801225, -0.089088894,  -0.17156231,   0.78059083,  0.080324806,   0.25679433,
  -0.32606214,  -0.18968545,   0.51530373,   0.18277867] TensorShape(dimensions: [64, 10])


In [ ]:
let yb = yTrain[0..<bs]
let loss = softmaxCrossEntropy(logits: preds, labels: yb)

In [ ]:
print(accuracy(preds, yb))

0.0625


In [ ]:
let lr:Float = 0.5   // learning rate
let epochs = 1      // how many epochs to train for

Then we can go

In [ ]:
let (loss, grads) = model.valueWithGradient { model -> TF in
    let preds = model(xb)
    return softmaxCrossEntropy(logits: preds, labels: yb)
}

Loop by hand

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/bs){
        let startIdx = i * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = model.valueWithGradient { model -> TF in
            let preds = model(xb)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        model.layer1.weight -= lr * grads.layer1.weight
        model.layer1.bias   -= lr * grads.layer1.bias
        model.layer2.weight -= lr * grads.layer2.weight
        model.layer2.bias   -= lr * grads.layer2.bias
    }
}

In [ ]:
let preds = model(xValid)
accuracy(preds, yValid)

0.9056


86% in one epoch, not too bad!

Naming all the parameters is a bit boring. We can use `AllDifferentiableVariables` objects to access them all.

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/bs){
        let startIdx = i * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = model.valueWithGradient { model -> TF in
            let preds = model(xb)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        var parameters = model.allDifferentiableVariables
        for kp in parameters.keyPaths{ 
            parameters[keyPath: kp] -= lr * grads[keyPath: kp]
        }
    }
}

Then we can use a S4TF optimizer to do the step for us.

In [ ]:
let optimizer = SGD(for: model, learningRate: lr)

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/bs){
        let startIdx = i * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = model.valueWithGradient { model -> TF in
            let preds = model(xb)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grads)
    }
}

## Dataset

We can create a swift `Dataset` from our arrays. It will automatically batch things for us.

In [ ]:
// export
public struct DataBatch<Inputs: Differentiable & TensorGroup, Labels: TensorGroup>: TensorGroup {
    public var xb: Inputs
    public var yb: Labels    
    
    public init(xb: Inputs, yb: Labels){ (self.xb,self.yb) = (xb,yb) }
}

In [ ]:
let train_ds:Dataset<DataBatch> = Dataset(elements:DataBatch(xb:xTrain, yb:yTrain)).batched(bs)

In [ ]:
for epoch in 1...epochs{
    for batch in train_ds{
        let (loss, grads) = model.valueWithGradient { model -> TF in
            let preds = model(batch.xb)
            return softmaxCrossEntropy(logits: preds, labels: batch.yb)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grads)
    }
}

This `Dataset` can also do the shuffle for us:

In [ ]:
for epoch in 1...epochs{
    for batch in train_ds.shuffled(sampleCount: 60000, randomSeed: 42){
        let (loss, grads) = model.valueWithGradient { model -> TF in
            let preds = model(batch.xb)
            return softmaxCrossEntropy(logits: preds, labels: batch.yb)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grads)
    }
}

### Training loop

In [ ]:
public func train<Opt: Optimizer, Label:TensorGroup>(
    _ model: inout Opt.Model,
    on ds: Dataset<DataBatch<Opt.Model.Input, Label>>,
    using opt: inout Opt,
    lossFunc: @escaping @differentiable (Opt.Model.Output, @nondiff Label) -> Tensor<Opt.Scalar>
) where Opt.Model: Layer,
        Opt.Model.Input: TensorGroup,
        Opt.Model.CotangentVector == Opt.Model.AllDifferentiableVariables,
        Opt.Scalar: TensorFlowFloatingPoint
{
    for batch in ds {
        let (loss, 𝛁model) = model.valueWithGradient { model -> Tensor<Opt.Scalar> in 
            let pred = model(batch.xb)                                      
            return lossFunc(pred, batch.yb)
        }
        opt.update(&model.allDifferentiableVariables, along: 𝛁model)
    }
}

In [ ]:
var model = MyModel(nIn: m, nHid: nHid, nOut: Int(c))
var optimizer = SGD(for: model, learningRate: lr)

In [ ]:
train(&model, on: train_ds, using: &optimizer, lossFunc: softmaxCrossEntropy)

In [ ]:
let preds = model(xValid)
accuracy(preds, yValid)

0.9052


### Export

In [ ]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"03_minibatch_training.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
